# Imports

In [153]:
import os
from collections import defaultdict, Counter

In [3]:
import pandas as pd

In [4]:
pd.set_option('max_columns', 100)

In [5]:
%matplotlib inline

In [6]:
os.getcwd()

'C:\\Users\\Dasha\\Documents\\GitHub\\find-political-donors'

In [7]:
os.listdir('.')

['.git',
 '.gitignore',
 '.ipynb_checkpoints',
 'Cleaning the input file.ipynb',
 'input',
 'insight_testsuite',
 'output',
 'README.md',
 'run.sh',
 'src']

In [8]:
os.listdir('input/')

['itcont.txt', 'README.md']

In [9]:
os.listdir('insight_testsuite/tests/test_1/input/')

['itcont.txt']

# Load data

## Column names

In [10]:
column_names = [
    'CMTE_ID',
    'AMNDT_IND',
    'RPT_TP',
    'TRANSACTION_PGI',
    'IMAGE_NUM',
    'TRANSACTION_TP',
    'ENTITY_TP',
    'NAME',
    'CITY',
    'STATE',
    'ZIP_CODE',
    'EMPLOYER',
    'OCCUPATION',
    'TRANSACTION_DT',
    'TRANSACTION_AMT',
    'OTHER_ID',
    'TRAN_ID',
    'FILE_NUM',
    'MEMO_CD',
    'MEMO_TEXT',
    'SUB_ID'
]
usecols = ['CMTE_ID', 'ZIP_CODE', 'TRANSACTION_DT', 'TRANSACTION_AMT', 'OTHER_ID']
dtype = {'ZIP_CODE': str, 'TRANSACTION_DT': str}

In [11]:
df1 = pd.read_csv('input/itcont.txt', sep='|', nrows=100000, names=column_names, usecols=usecols, dtype=dtype)

In [12]:
df1.head()

,CMTE_ID,ZIP_CODE,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID
0,C00629618,90017,01032017,40,H6CA34245
1,C00177436,330375267,01042017,5000,NaN
2,C00177436,307502818,01312017,230,NaN
3,C00177436,040976952,01312017,384,NaN
4,C00177436,300047357,01312017,384,NaN


In [13]:
df2 = pd.read_csv(
    'insight_testsuite/tests/test_1/input/itcont.txt', sep='|', names=column_names, usecols=usecols, dtype=dtype)

In [14]:
df2

,CMTE_ID,ZIP_CODE,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID
0,C00629618,90017,01032017,40,H6CA34245
1,C00177436,300047357,01312017,384,NaN
2,C00384818,028956146,01122017,250,NaN
3,C00177436,307502818,01312017,230,NaN
4,C00177436,041051896,01312017,384,NaN
5,C00384818,028956146,01122017,333,NaN
6,C00177436,041051935,01312017,384,NaN


# Filter rows

In [ ]:
PI = 3.14

In [106]:
def validate_transaction_dt(transaction_dt):
    month = transaction_dt[:2]
    day = transaction_dt[2:4]
    year = transaction_dt[4:]
    is_valid = (1 <= int(month) <= 12) and (1 <= int(day) <= 31) and (1000 <= int(year) <= 2020)
    return is_valid

In [97]:
df1_filtered = df1.copy()

In [98]:
before = len(df1_filtered)
df1_filtered = df1_filtered[df1_filtered.OTHER_ID.isnull()]
print(f"Lost {before - len(df1_filtered)} rows")

Lost 13016 rows


In [99]:
df1_filtered['ZIP_CODE_5CHAR'] = df1_filtered['ZIP_CODE'].str[:5]

In [100]:
df1_filtered['ZIP_VALID'] = df1_filtered.ZIP_CODE_5CHAR.str.len() == 5
print(f"Num invalid zip codes: {(~df1_filtered['ZIP_VALID']).sum()}")

Num invalid zip codes: 387


In [113]:
df1_filtered['DATE_VALID'] = df1_filtered.TRANSACTION_DT.apply(validate_transaction_dt)
print(f"Num invalid dates: {(~df1_filtered['DATE_VALID']).sum()}")

Num invalid dates: 0


In [101]:
before = len(df1_filtered)
df1_filtered = df1_filtered[df1_filtered.CMTE_ID.notnull()]
print(f"Lost {before - len(df1_filtered)} rows")

Lost 0 rows


In [102]:
before = len(df1_filtered)
df1_filtered = df1_filtered[df1_filtered.TRANSACTION_AMT.notnull()]
print(f"Lost {before - len(df1_filtered)} rows")

Lost 0 rows


# Output

In [186]:
class RollingMedian:
    
    def __init__(self):
        self._data = []
        
    def add(self, item):
        self._data.append(item)
        self._data.sort()
    
    @property
    def median(self):
        if len(self._data) % 2 == 0:
            return (self._data[len(self._data) // 2 - 1] + self._data[len(self._data) // 2]) / 2
        else:
            return self._data[len(self._data) // 2]

In [253]:
def round_amount(amount):
    if pd.isnull(amount):
        return None
    else:
        return int(amount) + int(amount % 1 >= 0.5)

In [210]:
run_med = defaultdict(RollingMedian)
run_count = Counter()
run_total = Counter()


def increment_run_med(key, value):
    rolling_median = run_med[key]
    rolling_median.add(value)
    return rolling_median.median

def increment_run_count(key):
    run_count[key] += 1
    return run_count[key]

def increment_run_total(key, value):
    run_total[key] += value
    return run_total[key]

In [211]:
df1_filtered.loc[df1_filtered['ZIP_VALID'], 'RUN_MED_ZIP'] = df1_filtered.loc[df1_filtered['ZIP_VALID'], :].apply(
    lambda s: increment_run_med((s['CMTE_ID'], s['ZIP_CODE_5CHAR']), s['TRANSACTION_AMT']), axis=1)

df1_filtered.loc[df1_filtered['ZIP_VALID'], 'RUN_COUNT_ZIP'] = df1_filtered.loc[df1_filtered['ZIP_VALID'], :].apply(
    lambda s: increment_run_count((s['CMTE_ID'], s['ZIP_CODE_5CHAR'])), axis=1)

df1_filtered.loc[df1_filtered['ZIP_VALID'], 'RUN_TOTAL_ZIP'] = df1_filtered.loc[df1_filtered['ZIP_VALID'], :].apply(
    lambda s: increment_run_total((s['CMTE_ID'], s['ZIP_CODE_5CHAR']), s['TRANSACTION_AMT']), axis=1)

In [212]:
df1_filtered.loc[df1_filtered['DATE_VALID'], 'RUN_MED_DATE'] = df1_filtered.loc[df1_filtered['DATE_VALID'], :].apply(
    lambda s: increment_run_med((s['CMTE_ID'], s['TRANSACTION_DT']), s['TRANSACTION_AMT']), axis=1)

df1_filtered.loc[df1_filtered['DATE_VALID'], 'RUN_COUNT_DATE'] = df1_filtered.loc[df1_filtered['DATE_VALID'], :].apply(
    lambda s: increment_run_count((s['CMTE_ID'], s['TRANSACTION_DT'])), axis=1)

df1_filtered.loc[df1_filtered['DATE_VALID'], 'RUN_TOTAL_DATE'] = df1_filtered.loc[df1_filtered['DATE_VALID'], :].apply(
    lambda s: increment_run_total((s['CMTE_ID'], s['TRANSACTION_DT']), s['TRANSACTION_AMT']), axis=1)

In [254]:
df1_filtered.RUN_MED_DATE = df1_filtered.RUN_MED_DATE.apply(round_amount)

In [257]:
df1_filtered.RUN_MED_ZIP = df1_filtered.RUN_MED_ZIP.apply(round_amount)

In [258]:
df1_filtered.head()

,CMTE_ID,ZIP_CODE,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,ZIP_CODE_5CHAR,ZIP_VALID,DATE_VALID,RUN_MED_ZIP,RUN_COUNT_ZIP,RUN_TOTAL_ZIP,RUN_MED_DATE,RUN_COUNT_DATE,RUN_TOTAL_DATE
1,C00177436,330375267,01042017,5000,NaN,33037,True,True,5000.0,1.0,5000.0,5000,1,5000
2,C00177436,307502818,01312017,230,NaN,30750,True,True,230.0,1.0,230.0,230,1,230
3,C00177436,040976952,01312017,384,NaN,04097,True,True,384.0,1.0,384.0,307,2,614
4,C00177436,300047357,01312017,384,NaN,30004,True,True,384.0,1.0,384.0,384,3,998
5,C00177436,020251500,01312017,230,NaN,02025,True,True,230.0,1.0,230.0,307,4,1228


# Save output

In [270]:
columns = ['CMTE_ID', 'ZIP_CODE_5CHAR', 'RUN_MED_ZIP', 'RUN_COUNT_ZIP', 'RUN_TOTAL_ZIP']
dtypes = {
    'RUN_MED_ZIP': int,
    'RUN_COUNT_ZIP': int,
    'RUN_TOTAL_ZIP': int,
}
df1_zip_out = df1_filtered[df1_filtered['ZIP_VALID']][columns].astype(dtypes)

In [271]:
columns = ['CMTE_ID', 'TRANSACTION_DT', 'RUN_MED_DATE', 'RUN_COUNT_DATE', 'RUN_TOTAL_DATE']
dtypes = {
    'RUN_MED_DATE': int,
    'RUN_COUNT_DATE': int,
    'RUN_TOTAL_DATE': int,
}
df1_date_out = df1_filtered[df1_filtered['DATE_VALID']][columns].astype(dtypes)

In [287]:
df1_date_out['year'] = df1_date_out['TRANSACTION_DT'].str[4:8].astype(int)

In [288]:
df1_date_out['month'] = df1_date_out['TRANSACTION_DT'].str[2:4].astype(int)

In [289]:
df1_date_out['day'] = df1_date_out['TRANSACTION_DT'].str[0:2].astype(int)

In [290]:
df1_date_out.head()

,CMTE_ID,TRANSACTION_DT,RUN_MED_DATE,RUN_COUNT_DATE,RUN_TOTAL_DATE,year,month,day
1,C00177436,01042017,5000,1,5000,2017,4,1
2,C00177436,01312017,230,1,230,2017,31,1
3,C00177436,01312017,307,2,614,2017,31,1
4,C00177436,01312017,384,3,998,2017,31,1
5,C00177436,01312017,307,4,1228,2017,31,1


In [291]:
df1_date_out.dtypes

CMTE_ID           object
TRANSACTION_DT    object
RUN_MED_DATE       int32
RUN_COUNT_DATE     int32
RUN_TOTAL_DATE     int32
year               int32
month              int32
day                int32
dtype: object

In [293]:
df1_date_out.sort_values(['CMTE_ID', 'year', 'month', 'day'])

,CMTE_ID,TRANSACTION_DT,RUN_MED_DATE,RUN_COUNT_DATE,RUN_TOTAL_DATE,year,month,day
93906,C00000059,02282017,416,1,416,2017,28,2
93907,C00000059,02282017,416,2,832,2017,28,2
16715,C00000059,01312017,416,1,416,2017,31,1
16716,C00000059,01312017,416,2,832,2017,31,1
36524,C00000422,02012017,100,1,100,2017,1,2
36417,C00000422,02032017,1000,1,1000,2017,3,2
36418,C00000422,02032017,750,2,1500,2017,3,2
36419,C00000422,02032017,500,3,1750,2017,3,2
36420,C00000422,02032017,375,4,1958,2017,3,2
36421,C00000422,02032017,500,5,2958,2017,3,2


In [278]:
df1_zip_out.to_csv('output/medianvals_by_zip.txt', sep='|', index=False, header=False)

In [279]:
df1_date_out.to_csv('output/medianvals_by_date.txt', sep='|', index=False, header=False)

# Scrap

In [103]:
df1_filtered

,CMTE_ID,ZIP_CODE,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,ZIP_CODE_5CHAR,ZIP_VALID
1,C00177436,330375267,01042017,5000,NaN,33037,True
2,C00177436,307502818,01312017,230,NaN,30750,True
3,C00177436,040976952,01312017,384,NaN,04097,True
4,C00177436,300047357,01312017,384,NaN,30004,True
5,C00177436,020251500,01312017,230,NaN,02025,True
6,C00177436,041051896,01312017,384,NaN,04105,True
7,C00177436,041051935,01312017,384,NaN,04105,True
8,C00177436,040424132,01312017,384,NaN,04042,True
9,C00177436,041051972,01312017,384,NaN,04105,True
10,C00177436,040722246,01312017,260,NaN,04072,True


In [104]:
temp = df1.merge(
    df1_filtered[['CMTE_ID']], left_index=True, right_index=True, suffixes=('', '_filtered'), how='left')
temp[
    (temp['CMTE_ID_filtered'].isnull()) &
    (temp['OTHER_ID'].isnull())
]

,CMTE_ID,ZIP_CODE,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,CMTE_ID_filtered


In [29]:
df1['TRANSACTION_DT'].apply(validate_transaction_dt).sum()

90043

In [35]:
df1[~df1.TRANSACTION_DT.apply(validate_transaction_dt)]

,CMTE_ID,ZIP_CODE,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID


In [30]:
df_filtered = df1[df1.TRANSACTION_DT.apply(validate_transaction_dt)]

In [31]:
df_filtered.head()

,CMTE_ID,ZIP_CODE,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID
0,C00629618,90017,01032017,40,H6CA34245
1,C00177436,330375267,01042017,5000,NaN
20,C00384818,441391855,01122017,250,NaN
21,C00384818,028956146,01122017,250,NaN
22,C00384818,028654257,01122017,333,NaN


In [17]:
df1['TRANSACTION_DT'].tail()

99995    02082017
99996    02122017
99997    02152017
99998    02192017
99999    02192017
Name: TRANSACTION_DT, dtype: object

In [124]:
df1_filtered

,CMTE_ID,ZIP_CODE,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,ZIP_CODE_5CHAR
1,C00177436,330375267,01042017,5000,NaN,33037
2,C00177436,307502818,01312017,230,NaN,30750
3,C00177436,040976952,01312017,384,NaN,04097
4,C00177436,300047357,01312017,384,NaN,30004
5,C00177436,020251500,01312017,230,NaN,02025
6,C00177436,041051896,01312017,384,NaN,04105
7,C00177436,041051935,01312017,384,NaN,04105
8,C00177436,040424132,01312017,384,NaN,04042
9,C00177436,041051972,01312017,384,NaN,04105
10,C00177436,040722246,01312017,260,NaN,04072


In [ ]:
df3 = 